<a href="https://colab.research.google.com/github/adanfernandez/Angular-training/blob/master/Ejercicio1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Primer ejercicio
## Preparación del entorno
Preparamos el entorno instalando las dependencias necesarias.


In [ ]:
!pip install spacy==3.0.5
!pip install ndjson

     |████████████████████████████████| 12.8MB 16.2MB/s 
     |████████████████████████████████| 1.1MB 35.6MB/s 
     |████████████████████████████████| 51kB 8.0MB/s 
     |████████████████████████████████| 9.1MB 47.5MB/s 
     |████████████████████████████████| 460kB 57.1MB/s 
     |████████████████████████████████| 122kB 49.4MB/s 
  Created wheel for smart-open: filename=smart_open-3.0.0-cp37-none-any.whl size=107098 sha256=d44338cd40d6aad9f5000a19c40c9f99c1e7d02ec5df25ec6fbc62be2d7da6a8
  Stored in directory: /root/.cache/pip/wheels/18/88/7c/f06dabd5e9cabe02d2269167bcacbbf9b47d0c0ff7d6ebcb78
Successfully built smart-open
  Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Found 

Instalamos el modelo que se usará en spaCy. Se instalará el modelo grande.

In [ ]:
!python -m spacy download es_core_news_lg

2021-05-25 22:04:57.848176: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 569.7MB 26kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_lg')


## Descarga de los datos de la máquina virtual
Importamos el archivo de datos desde el enlace Google Drive otorgado por el profesor de la asignatura. Para el primer ejercicio, que es el primero en el que nos encontramos, descargaremos el archivo con los datos procedentes de la API de twitter. El objetivo será limpiarlos.
La descarga de los tweets tardará más de dos minutos en producirse, realizándose en un directorio específico para el ejercicio 1.

In [ ]:
!mkdir ejercicio1
%cd ejercicio1

!gdown --id 1yBH-MYb3FapTWQ9ps3dkSjhaGOZKfNMK
!unzip tweets.zip


  inflating: 20130204/10.json        
  inflating: 20130204/11.json        
  inflating: 20130204/12.json        
  inflating: 20130204/13.json        
  inflating: 20130204/14.json        
  inflating: 20130204/15.json        
  inflating: 20130204/16.json        
  inflating: 20130204/17.json        
  inflating: 20130204/18.json        
  inflating: 20130204/19.json        
  inflating: 20130204/20.json        
  inflating: 20130204/21.json        
  inflating: 20130204/22.json        
  inflating: 20130204/23.json        
   creating: 20130205/
  inflating: 20130205/00.json        
  inflating: 20130205/01.json        
  inflating: 20130205/02.json        
  inflating: 20130205/03.json        
  inflating: 20130205/04.json        
  inflating: 20130205/05.json        
  inflating: 20130205/06.json        
  inflating: 20130205/07.json        
  inflating: 20130205/08.json        
  inflating: 20130205/09.json        
  inflating: 20130205/10.json        
  inflating: 20130205/11.j

Eliminamos el fichero .zip para no agotar el espacio disponible.

In [ ]:
!rm tweets.zip

Se cargan los datos en memoria para posteriormente procesarlos. 

In [ ]:
import os
def cargar_tweets(directorio):
    lista = {}
    with os.scandir(directorio) as subdirectorios:
        for subdirectorio in subdirectorios:
            lista[directorio+subdirectorio.name] = os.listdir(directorio+subdirectorio.name)
    return lista

elementos_memoria = cargar_tweets("/content/ejercicio1/")
print(elementos_memoria)

{'/content/ejercicio1/20130129': ['10.json', '06.json', '09.json', '16.json', '20.json', '00.json', '23.json', '17.json', '08.json', '19.json', '12.json', '07.json', '02.json', '13.json', '05.json', '03.json', '22.json', '14.json', '11.json', '01.json', '15.json', '04.json', '18.json', '21.json'], '/content/ejercicio1/20130211': ['10.json', '06.json', '09.json', '16.json', '20.json', '00.json', '23.json', '17.json', '08.json', '19.json', '12.json', '07.json', '02.json', '13.json', '05.json', '03.json', '22.json', '14.json', '11.json', '01.json', '15.json', '04.json', '18.json', '21.json'], '/content/ejercicio1/20130120': ['10.json', '06.json', '09.json', '16.json', '20.json', '00.json', '23.json', '17.json', '08.json', '19.json', '12.json', '07.json', '02.json', '13.json', '05.json', '03.json', '22.json', '14.json', '11.json', '01.json', '15.json', '04.json', '18.json', '21.json'], '/content/ejercicio1/20130124': ['10.json', '06.json', '09.json', '16.json', '20.json', '00.json', '23.js

## Limpieza de textos
Se usará la librería ndjson https://pypi.org/project/ndjson/ para leer los archivos, además de la biblioteca io.
Por otro lado, se usará el paquete spacy para obtener tanto los tokens como los sintagmas nominales de los tweets escritos en castellano.

In [ ]:
import json
import io
import ndjson
import spacy
from tqdm import tqdm

def limpiar_tweets(lista):
    json_data = []
    nlp = spacy.load("es_core_news_lg")
    for subdirectorio in lista:
        for fichero in lista[subdirectorio]:
          ruta = subdirectorio + "/" + fichero
          get_tweets(ruta, nlp)
        break

def get_tweets(ruta, nlp):
  english_tweets = []
  tweets = []
  with open(ruta) as f:
    datos = ndjson.load(f)
    for entrada in datos:
      if 'lang' in entrada and entrada['lang']=="es" or 'lang' not in entrada and entrada['user']['lang'] == 'es':
        hashtags = []
        for hashtag in entrada['entities']['hashtags']:
          hashtags.append(hashtag['text'])
        tweet_append = {
            "created_at": entrada['created_at'],
            "id_str": entrada['id_str'],
            "text": entrada['text'],
            "user": {
                "screen_name": entrada['user']['screen_name'],
                "description": entrada['user']['description']
            },
            "hashtags": hashtags,
            "urls": entrada['entities']['urls']
        }
        doc = nlp(entrada['text'])
        noum_chunks = []
        for chunk in doc.noun_chunks:
          noum_chunks.append(chunk.root.text)
        tweet_append['noum_chunks'] = noum_chunks
        tokens = [token.lemma_.lower() for token in doc if not token.is_punct and not token.is_stop]
        tweet_append['tokens'] = tokens
        tweets.append(tweet_append)
    return tweets

tweets_limpios = limpiar_tweets(elementos_memoria)

KeyboardInterrupt: ignored